<a href="https://colab.research.google.com/github/BClarke94/MSCAI/blob/main/Bitcoin-Buy/Sell-Indicator-%20Project-Machine-Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CA01_MACHINE LEARNING

### **Machine Learning for Cryptocurrencies**

Robert Clarke
Anders Kiss

# **Data Understanding**

### Data Source API** 
## *Binance API*
https://www.binance.com/en/support/faq/360002502072



In [ ]:
pip install python-binance


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from binance.client import Client
#client = Client(api_key, api_secret)

api_key = "aHiHo6mcN8QvXJNwnbqnFLaFCSyIpxNidUu2VbYK64NZ2GczTwfNi733gVScmGbj"
api_secret = "x28t2T69sR6R8eNxAu89Y4JEtQ3wp7ZnEWVMnh4MA6lcHo0dLXrbCZzXqcdbWjVq"

client = Client(api_key, api_secret, {"verify": False, "timeout": 20})


# Data Preparation

##*italicized text*# crypto symbol (e.g. BTC for Bitcoin or ETH for Ethereum)


In [ ]:
from datetime import date, datetime, timedelta
from time import time

In [ ]:
# API call takes date value as a string.

# ESTABLISH timestamps for: now, today, yesterday, one week ago, six months ago, one year ago

today = datetime.now()
print(today)

yesterday = today - timedelta(1)
oneWeekAgo = today - timedelta(7)
oneMonthAgo = today - timedelta(30)
twoMonthAgo = today - timedelta(60)

# CONVERT date from datetime to string in proper format for API call
today = today.strftime('%d %b, %Y').title()
yesterday = yesterday.strftime('%d %b, %Y').title()
oneWeekAgo = oneWeekAgo.strftime('%d %b, %Y').title()
oneMonthAgo = oneMonthAgo.strftime('%d %b, %Y').title()
twoMonthAgo = twoMonthAgo.strftime('%d %b, %Y').title()

print("- - - - - - - - - - -")
print("Today's date is: {}\nYesterday's date was: {}".format(today, yesterday))
print("- - - - - - - - - - -")
print("The time one week ago was: ",oneWeekAgo)
print("The time 30 days ago was: ",oneMonthAgo)

2021-12-10 10:38:58.574157
- - - - - - - - - - -
Today's date is: 10 Dec, 2021
Yesterday's date was: 09 Dec, 2021
- - - - - - - - - - -
The time one week ago was:  03 Dec, 2021
The time 30 days ago was:  10 Nov, 2021


In [ ]:
# GET THE CRYPTOCURRENCY SYMBOL that the user wants a BUY/SELL/STAY NEUTRAL prediction for
symbol = input("Enter a cryptocurrency ticker symbol (three to four characters like BTC or ETH): ")
print("You entered: {} \nThe current date is: {} \nThe date two months ago was: {}\n".format(symbol, today, twoMonthAgo))

# FORMAT symbol to binance API format
symbol = ''.join((symbol, 'USDT'))
print(symbol)

Enter a cryptocurrency ticker symbol (three to four characters like BTC or ETH): BTC
You entered: BTC 
The current date is: 10 Dec, 2021 
The date two months ago was: 11 Oct, 2021

BTCUSDT


## Call Binance API with python-binance wrapper

In [ ]:
# CALL API with values: $SYMBOL, $START_TIME, $TIME_NOW

klinesData = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, twoMonthAgo, today)

# ENSURE that array of 15-minute intervals is divisible by 4; otherwise, there is typically one additional value.
if len(klinesData) % 4 != 0: 
  arr_length = len(klinesData)
  print("Array of length {}, is not divisible by 4".format(arr_length))
  klinesData.pop()
  arr_length = len(klinesData)
  print("The new array has {} elements".format(arr_length))

print("- - - - - - - - - - -")
print(klinesData)
print(len(klinesData))

Array of length 5761, is not divisible by 4
The new array has 5760 elements
- - - - - - - - - - -
[[1633910400000, '54659.01000000', '54919.51000000', '54415.06000000', '54700.03000000', '1220.86012000', 1633911299999, '66756797.61155980', 33087, '632.19497000', '34573344.52148340', '0'], [1633911300000, '54700.03000000', '54800.25000000', '54524.87000000', '54716.01000000', '831.15214000', 1633912199999, '45412876.12087420', 24217, '422.19778000', '23067235.65682500', '0'], [1633912200000, '54716.01000000', '54950.00000000', '54695.37000000', '54946.57000000', '939.92544000', 1633913099999, '51566386.31244550', 25169, '484.22018000', '26563935.53088010', '0'], [1633913100000, '54946.57000000', '55180.00000000', '54874.09000000', '55159.93000000', '898.67939000', 1633913999999, '49437025.53830590', 24355, '425.39780000', '23405499.89969270', '0'], [1633914000000, '55159.93000000', '55462.67000000', '55066.69000000', '55436.52000000', '985.29769000', 1633914899999, '54440611.94964880', 

## Take kline (candlestick) and CREATE list of 15-minute closing price values
## CONVERT to float, APPEND to list

In [ ]:
# ISOLATE 15-minute closing price data in a list, CONVERT to float, APPEND to list
clpr_15m = [] # use this list to generate all other timeframe values
dates = [] # mainly for reference as the DataFrame index

for list in klinesData:
  clpr_15m.append(float(list[4]))
  dates.append(list[0])

print("- - - - - - - - - -")
print(clpr_15m)
print(len(clpr_15m))
print(type(clpr_15m[0]))
print("- - - - - - - - - -")
# print(dates[0])
print(dates)
print(len(dates))
print(type(dates[0]))

- - - - - - - - - -
[54700.03, 54716.01, 54946.57, 55159.93, 55436.52, 55513.32, 55497.41, 55334.02, 55364.53, 55372.44, 55486.84, 55799.99, 55739.72, 56114.99, 56398.13, 56507.09, 56534.12, 56606.61, 56520.0, 56460.0, 56419.0, 56349.05, 56342.81, 56528.93, 56630.55, 56541.16, 56680.99, 56678.63, 56775.01, 56677.72, 56734.41, 56789.76, 56667.98, 56381.03, 56416.29, 56555.0, 56609.13, 56565.91, 56532.17, 56491.79, 56428.77, 56504.45, 56367.76, 56177.65, 56038.58, 56181.42, 56582.15, 56407.51, 56246.88, 56344.49, 56453.36, 56462.83, 56708.86, 56988.55, 57269.99, 57222.78, 57213.07, 57392.8, 57483.3, 57447.99, 57335.02, 57409.38, 57530.83, 57490.18, 57603.61, 57475.36, 57331.51, 57280.0, 57446.99, 57442.11, 57423.53, 57418.99, 57434.53, 57480.01, 57318.4, 57343.26, 57499.96, 57577.06, 57721.02, 57488.33, 56961.37, 57312.6, 57426.57, 57327.03, 57098.07, 57147.89, 57300.99, 57156.54, 57246.78, 57223.43, 57289.05, 57170.4, 57185.15, 57300.0, 57235.4, 57471.35, 56880.61, 56930.62, 56760.87, 5

## CREATE lists for 1h, 4h, and 1d data

### Values are generated from the last value in the previous time interval's closing price.

Example:
The 1 hour list contains four duplicate values based on the closing price of the last 15-minute time intervals for that hour.

In [ ]:
# RETURN a list of the last 15-minute closing price for a given hour
import numpy as np

# ADD data for 1h, 4h, 1d timeframes based on 15 minute data
clpr_1h = []
clpr_4h = []
clpr_1d = []

# CREATE 1 Hour timeframe data
list_15m_to_1hr = clpr_15m[3::4]
list_15m_to_1hr[:10] # returning 1 hour data as expected
clpr_1h = np.repeat(list_15m_to_1hr, 4).tolist() # working as expected
clpr_1h[:16] # TEST, the list is working as expected
# print(type(clpr_1h[1])) # <class 'float'>

# CREATE 4 Hour timeframe data
list_15m_to_4hr = clpr_15m[15::16]
list_15m_to_4hr[:50] # returning 4 hour data as expected
clpr_4h = np.repeat(list_15m_to_4hr, 16).tolist() # working as expected
# clpr_4h[:64] # working as expected

# CREATE 1 Day timeframe data
list_15m_to_1d = clpr_15m[95::96]
list_15m_to_1d # returning 1 day data as expected => ... EX: Each day has 96x 15-minute intervals; len(clpr_15m) = 961; (961/96) = 10.0104
# len(list_15m_to_1d) # returns 10; working as expected
clpr_1d = np.repeat(list_15m_to_1d, 96).tolist() # working as expected
# clpr_1d[:100] # working as expected

 CONFIRM that all lists are of equal length before adding to DataFrame

(applies for any timeframe)

In [ ]:
print(clpr_15m)
print(list_15m_to_1hr)
print("- - - - - - - - - ")

# CONFIRM that all lists are of equal length, in order to create a DataFrame
print(len(dates))
print(len(clpr_15m))
print(len(clpr_1h))
print(len(clpr_4h))
print(len(clpr_1d))

[54700.03, 54716.01, 54946.57, 55159.93, 55436.52, 55513.32, 55497.41, 55334.02, 55364.53, 55372.44, 55486.84, 55799.99, 55739.72, 56114.99, 56398.13, 56507.09, 56534.12, 56606.61, 56520.0, 56460.0, 56419.0, 56349.05, 56342.81, 56528.93, 56630.55, 56541.16, 56680.99, 56678.63, 56775.01, 56677.72, 56734.41, 56789.76, 56667.98, 56381.03, 56416.29, 56555.0, 56609.13, 56565.91, 56532.17, 56491.79, 56428.77, 56504.45, 56367.76, 56177.65, 56038.58, 56181.42, 56582.15, 56407.51, 56246.88, 56344.49, 56453.36, 56462.83, 56708.86, 56988.55, 57269.99, 57222.78, 57213.07, 57392.8, 57483.3, 57447.99, 57335.02, 57409.38, 57530.83, 57490.18, 57603.61, 57475.36, 57331.51, 57280.0, 57446.99, 57442.11, 57423.53, 57418.99, 57434.53, 57480.01, 57318.4, 57343.26, 57499.96, 57577.06, 57721.02, 57488.33, 56961.37, 57312.6, 57426.57, 57327.03, 57098.07, 57147.89, 57300.99, 57156.54, 57246.78, 57223.43, 57289.05, 57170.4, 57185.15, 57300.0, 57235.4, 57471.35, 56880.61, 56930.62, 56760.87, 56941.7, 56810.62, 57

## ADD all data to DataFrame

In [ ]:
# ADD existing lists to DataFrame
df = pd.DataFrame({'dates_15m': dates,\
                   'cl_pr_15m': clpr_15m, 'BB_UB_15m': '', 'SMA20_15m': '', 'BB_LB_15m': '', 'cat_15m': '',\
                   'cl_pr_1h': clpr_1h, 'BB_UB_1h': '', 'SMA20_1h': '', 'BB_LB_1h': '', 'cat_1h': '',\
                   'cl_pr_4h': clpr_4h, 'BB_UB_4h': '', 'SMA20_4h': '', 'BB_LB_4h': '', 'cat_4h': '',\
                   'cl_pr_1d': clpr_1d, 'BB_UB_1d': '', 'SMA20_1d': '', 'BB_LB_1d': '', 'cat_1d': ''})

df.set_index('dates_15m', inplace=True)

# CREATE new features for Bollinger Band Uppper and Lower Bounds, and Simple Moving Average for each timeframe
df['BB_UB_15m'] = df['cl_pr_15m'].rolling(20).mean() + 2*df['cl_pr_15m'].rolling(20).std()
df['BB_LB_15m'] = df['cl_pr_15m'].rolling(20).mean() - 2*df['cl_pr_15m'].rolling(20).std()
df['SMA20_15m'] = df['cl_pr_15m'].rolling(20).mean()

df['BB_UB_1h'] = df['cl_pr_1h'].rolling(20).mean() + 2*df['cl_pr_1h'].rolling(20).std()
df['BB_LB_1h'] = df['cl_pr_1h'].rolling(20).mean() - 2*df['cl_pr_1h'].rolling(20).std()
df['SMA20_1h'] = df['cl_pr_1h'].rolling(20).mean()

df['BB_UB_4h'] = df['cl_pr_4h'].rolling(20).mean() + 2*df['cl_pr_4h'].rolling(20).std()
df['BB_LB_4h'] = df['cl_pr_4h'].rolling(20).mean() - 2*df['cl_pr_4h'].rolling(20).std()
df['SMA20_4h'] = df['cl_pr_4h'].rolling(20).mean()

df['BB_UB_1d'] = df['cl_pr_1d'].rolling(20).mean() + 2*df['cl_pr_1d'].rolling(20).std()
df['BB_LB_1d'] = df['cl_pr_1d'].rolling(20).mean() - 2*df['cl_pr_1d'].rolling(20).std()
df['SMA20_1d'] = df['cl_pr_1d'].rolling(20).mean()


# 4x categorical variables:

# If the closing price is above the upper Bollinger Band upper bound                                        =>  1 (a strong "SELL" signal)
# If the closing price is below or equal to the upper Bollinger Band and above the Mean                     =>  2 (a weak "SELL" or "HOLD/STAY NEUTRAL" signal)
# If the closing price is below the Mean and above or equal to the lower Bollinger Band lower bound         =>  3 (a weak "BUY" or "HOLD/STAY NEUTRAL" signal)
# If the closing price is below the lower Bollinger Band                                                    =>  4 ( "BUY" signal)

# 15 Minute categorical variable 
df.loc[(df.cl_pr_15m > df.BB_UB_15m),'cat_15m']='1'
df.loc[((df.cl_pr_15m >= df.SMA20_15m) & (df.cl_pr_15m <= df.BB_UB_15m)),'cat_15m']='2'
df.loc[((df.cl_pr_15m < df.SMA20_15m) & (df.cl_pr_15m > df.BB_LB_15m)),'cat_15m']='3'
df.loc[(df.cl_pr_15m < df.BB_LB_15m),'cat_15m']='4'

# 1 Hour categorical variable 
df.loc[(df.cl_pr_1h > df.BB_UB_1h),'cat_1h']='1'
df.loc[((df.cl_pr_1h >= df.SMA20_1h) & (df.cl_pr_1h <= df.BB_UB_1h)),'cat_1h']='2'
df.loc[((df.cl_pr_1h < df.SMA20_1h) & (df.cl_pr_1h > df.BB_LB_1h)),'cat_1h']='3'
df.loc[(df.cl_pr_1h < df.BB_LB_1h),'cat_1h']='4'

# 4 Hour categorical variable 
df.loc[(df.cl_pr_4h > df.BB_UB_4h),'cat_4h']='1'
df.loc[((df.cl_pr_4h >= df.SMA20_4h) & (df.cl_pr_4h <= df.BB_UB_4h)),'cat_4h']='2'
df.loc[((df.cl_pr_4h < df.SMA20_4h) & (df.cl_pr_4h > df.BB_LB_4h)),'cat_4h']='3'
df.loc[(df.cl_pr_4h < df.BB_LB_4h),'cat_4h']='4'

# 1 Day categorical variable with condition for when CLOSING PRICE equals SMA and UPPER BOUND
df.loc[(df.cl_pr_1d > df.BB_UB_1d),'cat_1d']='1'
df.loc[((df.cl_pr_1d >= df.SMA20_1d) & (df.cl_pr_1d <= df.BB_UB_1d)),'cat_1d']='2'
df.loc[((df.cl_pr_1d < df.SMA20_1d) & (df.cl_pr_1d > df.BB_LB_1d)),'cat_1d']='3'
df.loc[(df.cl_pr_1d < df.BB_LB_1d),'cat_1d']='4'

df = df.dropna()

df.head(10)


,cl_pr_15m,BB_UB_15m,SMA20_15m,BB_LB_15m,cat_15m,cl_pr_1h,BB_UB_1h,SMA20_1h,BB_LB_1h,cat_1h,cl_pr_4h,BB_UB_4h,SMA20_4h,BB_LB_4h,cat_4h,cl_pr_1d,BB_UB_1d,SMA20_1d,BB_LB_1d,cat_1d
dates_15m,,,,,,,,,,,,,,,,,,,,
1633927500000,56460.00,56963.631500,55710.4135,54457.195500,2,56460.00,56994.235920,55852.2060,54710.176080,2,56789.76,56795.634641,56563.6240,56331.613359,2,57471.35,57471.35,57471.35,57471.35,1
1633928400000,56419.00,56992.286224,55796.3620,54600.437776,2,56528.93,57052.036607,55920.6560,54789.275393,2,56789.76,56828.916386,56577.7575,56326.598614,2,57471.35,57471.35,57471.35,57471.35,1
1633929300000,56349.05,56982.891003,55878.0140,54773.136997,2,56528.93,57091.991214,55989.1060,54886.220786,2,56789.76,56857.692581,56591.8910,56326.089419,2,57471.35,57471.35,57471.35,57471.35,1
1633930200000,56342.81,56978.875618,55947.8260,54916.776382,2,56528.93,57112.654807,56057.5560,55002.457193,2,56789.76,56882.679557,56606.0245,56329.369443,2,57471.35,57471.35,57471.35,57471.35,1
1633931100000,56528.93,57008.110499,56016.2760,55024.441501,2,56528.93,57111.224336,56126.0060,55140.787664,2,56789.76,56904.311842,56620.1580,56336.004158,2,57471.35,57471.35,57471.35,57471.35,1
1633932000000,56630.55,57064.615542,56075.9775,55087.339458,2,56678.63,57133.378114,56193.2365,55253.094886,2,56789.76,56922.851091,56634.2915,56345.731909,2,57471.35,57471.35,57471.35,57471.35,1
1633932900000,56541.16,57099.579477,56127.3695,55155.159523,2,56678.63,57131.681490,56260.4670,55389.252510,2,56789.76,56938.438301,56648.4250,56358.411699,2,57471.35,57471.35,57471.35,57471.35,1
1633933800000,56680.99,57141.234091,56186.5485,55231.862909,2,56678.63,57099.772928,56327.6975,55555.622072,2,56789.76,56951.118091,56662.5585,56373.998909,2,57471.35,57471.35,57471.35,57471.35,1
1633934700000,56678.63,57142.800615,56253.7790,55364.757385,2,56678.63,57023.516570,56394.9280,55766.339430,2,56789.76,56960.845842,56676.6920,56392.538158,2,57471.35,57471.35,57471.35,57471.35,1


In [ ]:
df.shape

(5741, 20)

In [ ]:
df.value_counts("cat_1d")

cat_1d
1    2471
2    2373
3     573
4     324
dtype: int64

# Model Creation

## Decision Tree Classifier Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score
dct = DecisionTreeClassifier(criterion='entropy')

X=df.drop(['cat_4h'],axis=1)
y=df['cat_4h']

#oversample = RandomOverSampler(sampling_strategy='minority')
#undersample = RandomUnderSampler(sampling_strategy='majority')
#X_over, y_over = oversample.fit_resample(X, y)
#X_under, y_under = undersample.fit_resample(X, y)
accS=[]
recallS=[]
percS=[]
validationSet=[]
#X_over, y_over = oversample.fit_resample(X, y)

for x in range(100):
  #X_train,X_test,y_train,y_test = train_test_split(X_over,y_over,test_size=0.3)
  X_train,X_rem,y_train,y_rem = train_test_split(X,y,test_size=0.3)
  X_test,X_valid,y_test,y_valid = train_test_split(X,y,test_size=0.5)
  dct.fit(X_train, y_train)
  y_pred = dct.predict(X_test)
  y_Vpred = dct.predict(X_valid)
  acc=accuracy_score(y_test, y_pred)
  rec=recall_score(y_test, y_pred, average='weighted')
  perc=precision_score(y_test, y_pred, average='weighted')
  validPrec = precision_score(y_valid, y_Vpred, average='weighted')
  accS.append(acc)
  recallS.append(rec)
  percS.append(perc)
  validationSet.append(validPrec)

averageRecall = np.mean(recallS)
averagePerc = np.mean(percS)  
print("Accuracy Score:", np.mean(accS))
print("Recall Score:", np.mean(recallS))
print("Precision Score:", np.mean(percS))
print("F1 Score:", 2*((averageRecall*averagePerc)/(averagePerc + averageRecall)))
print("Validation Test Set Average Percision:", np.mean(validationSet))

Accuracy Score: 0.9674634146341462
Recall Score: 0.9674634146341462
Precision Score: 0.9674878992536192
F1 Score: 0.9674756567889701
Validation Test Set Average Percision: 0.9677464516652994


## SVM Models:

In [ ]:
from sklearn.metrics import  accuracy_score
import numpy as np
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
undersample = RandomUnderSampler(sampling_strategy='majority')
X=df.drop(['cat_4h'],axis=1)
y=df['cat_4h']  
recallS = []
percS = []
accS=[]
valid = []
svmL = SVC(kernel='linear')
X_under, y_under = undersample.fit_resample(X, y)



for x in range(100): 

  X_train,X_test,y_train,y_test=train_test_split(X_under,y_under,test_size=0.3)
  scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
  X_trainSVM = scaling.transform(X_train)
  X_testSVM = scaling.transform(X_test) 

  svmL.fit(X_trainSVM, y_train)
  pred_svc = svmL.predict(X_testSVM)
  rec = recall_score(y_test, pred_svc, average='weighted')
  accu=accuracy_score(y_test, pred_svc)   
  perc = precision_score(y_test, pred_svc, average='weighted')
  recallS.append(rec)
  accS.append(np.mean(accu))
  percS.append(np.mean(perc))
 
averageRecall = np.mean(recallS)
averagePerc = np.mean(percS)
print("Accuracy", np.mean(accS))
print("Recall", averageRecall)
print("Percision", np.mean(percS))
print("F1 Score:", 2*((averageRecall*averagePerc)/(averagePerc + averageRecall)))


Accuracy 0.7847140039447731
Recall 0.7847140039447731
Percision 0.6806221742589955
F1 Score: 0.7289709480742824


## Adaboost Model:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score, recall_score
adb = AdaBoostClassifier(n_estimators=100, learning_rate=1)
X=df.drop(['cat_4h'],axis=1)
y=df['cat_4h']
acAdabst=[]
acc=[]
percS=[]
recallS = []

#oversample = RandomOverSampler(sampling_strategy='minority')
#X_over, y_over = oversample.fit_resample(X, y)

for x in range(100):
  #X_train,X_test,y_train,y_test = train_test_split(X_over,y_over,test_size=0.3)
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
  adb.fit(X_train, y_train)
  y_pred = adb.predict(X_test)
  acc=accuracy_score(y_test, y_pred)
  rec = recall_score(y_test, y_pred, average='weighted')
  perc = precision_score(y_test, y_pred, average='weighted')
  acAdabst.append(acc)
  recallS.append(rec)
  percS.append(perc)


averageRecall = np.mean(recallS)
averagePerc = np.mean(percS)  

print("Accuracy", np.mean(acAdabst))
print("Recall", averageRecall)
print("Percision", averagePerc)
print("F1 Score:", 2*((averageRecall*averagePerc)/(averagePerc + averageRecall)))


Accuracy 0.7362217063261752
Recall 0.7362217063261752
Percision 0.712736339427375
F1 Score: 0.7242886921560244


# Prediction for today: 

In [ ]:
#@title
import pandas as pd
from datetime import date, datetime, timedelta
from time import time
from time import mktime
from binance.client import Client
import warnings

timeNow = datetime.utcnow()
timeNow = timeNow.replace(second=0, microsecond=0, minute=0)
timeNow = timeNow.strftime("%d/%m/%Y %H:%M:%S")
print(timeNow)

today = date.today()
oneWeek = timedelta(7)
oneWeekAgo = today - oneWeek
oneWeekAgo = oneWeekAgo.strftime('%d %b, %Y').title()
today = today.strftime('%d %b, %Y').title()

print(symbol)


10/12/2021 11:00:00
BTCUSDT


In [ ]:
#@title
klines_15m_past1day = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_15MINUTE)

# ENSURE that array of 15-minute intervals is divisible by 4; otherwise, there is typically one additional value.
if len(klines_15m_past1day) % 4 > 0: 
  arr_length = len(klines_15m_past1day)
  print("Array of length {}, is not divisible by 4".format(arr_length))
  klines_15m_past1day.pop()
  arr_length = len(klines_15m_past1day)
  print("The new array has {} elements".format(arr_length))

In [ ]:
#@title
# GET closing price, CONVERT to float, APPEND to list
clpr_15m = []
dates = []
dates_15m = []
for list in klines_15m_past1day:
  clpr_15m.append(float(list[4]))
  dates.append(list[0])
  dates_15m.append(float(list[0]))


var = dates_15m[0]

t = datetime.now()
unix_secs = mktime(t.timetuple())  

In [ ]:
#@title
# ADD data for 1h, 4h, 1d timeframes based on 15 minute data
clpr_1h = []
clpr_4h = []
clpr_1d = []

# tf = list(range(1, 97))

string_clpr_15m = []
for i in clpr_15m:
  string_clpr_15m.append(str(i))
splitSize = 4
split_4 = [string_clpr_15m[x:x+splitSize] for x in range(0, len(string_clpr_15m), splitSize)]
print(split_4)
print(len(split_4))

[['49487.0', '49371.57', '49464.64', '49584.29'], ['49525.05', '49441.24', '49289.18', '49366.04'], ['49351.39', '49467.25', '49474.55', '49027.46'], ['49051.0', '49209.31', '48990.84', '49073.46'], ['49290.59', '49350.2', '49233.73', '49344.63'], ['49354.2', '49150.89', '49242.58', '49397.71'], ['49467.11', '49465.84', '49382.34', '49315.62'], ['49180.01', '49395.02', '49297.52', '48910.62'], ['48959.49', '49043.09', '48926.21', '48560.99'], ['48458.52', '47869.98', '47951.57', '48604.2'], ['48297.13', '48428.68', '48479.81', '48608.51'], ['49027.72', '49157.48', '49262.21', '49243.41'], ['49139.89', '49200.01', '49089.18', '48610.98'], ['48586.68', '48800.0', '48902.45', '49007.27'], ['48543.78', '48861.22', '48814.66', '48676.0'], ['48887.64', '49105.15', '49158.83', '48867.44'], ['49212.38', '48925.88', '49214.1', '49505.96'], ['49188.4', '49201.37', '49396.33', '48855.32'], ['48692.27', '48638.61', '48743.61', '48862.07'], ['48457.61', '48530.12', '48804.19', '48944.91'], ['48852.

Creating prediction dataset

In [ ]:
#@title
import numpy as np

# CREATE 1 Hour timeframe data
list_15m_to_1hr = clpr_15m[3::4]
list_15m_to_1hr[:10] # returning 1 hour data as expected

# print(len(list_15m_to_1hr))

clpr_1h = np.repeat(list_15m_to_1hr, 4).tolist() # working as expected
clpr_1h[:16] # TEST, the list is working as expected
# print(type(clpr_1h[1])) # <class 'float'>

# CREATE 4 Hour timeframe data
list_15m_to_4hr = clpr_15m[15::16]
list_15m_to_4hr[:50] # returning 4 hour data as expected
clpr_4h = np.repeat(list_15m_to_4hr, 16).tolist() # working as expected
# clpr_4h[:64] # working as expected

# CREATE 1 Day timeframe data
list_15m_to_1d = clpr_15m[95::96]
list_15m_to_1d # returning 1 day data as expected => ... EX: Each day has 96x 15-minute intervals; len(clpr_15m) = 961; (961/96) = 10.0104
# len(list_15m_to_1d) # returns 10; working as expected
clpr_1d = np.repeat(list_15m_to_1d, 96).tolist() # working as expected


In [ ]:
#@title
cols = ['dates_15m', 'cl_pr_15m', 'BB_UB_15m', 'SMA20_15m', 'BB_LB_15m', 'cat_15m']
dfPred = pd.DataFrame(columns = cols)

for i in dates_15m:
    dfPred = dfPred.append({'dates_15m': str(i)},ignore_index=True)
for i in clpr_15m:
    dfPred = dfPred.append({'cl_pr_15m': str(i)},ignore_index=True)

dfPred.set_index('dates_15m', inplace=True)



In [ ]:
#@title
arSize= (len(clpr_1d))
arClose = len(clpr_15m)

clpr_15m = clpr_15m[arSize:]
clpr_4h = clpr_1h[arSize:]
clpr_1d = clpr_1h[arSize:]
clpr_1h = clpr_1h[arSize:]

dates = dates[arSize:arClose]

In [ ]:
#@title

dfPred = pd.DataFrame({'dates_15m': dates,\
                   'cl_pr_15m': clpr_15m, 'BB_UB_15m': '', 'SMA20_15m': '', 'BB_LB_15m': '', 'cat_15m': '',\
                   'cl_pr_1h': clpr_1h, 'BB_UB_1h': '', 'SMA20_1h': '', 'BB_LB_1h': '', 'cat_1h': '',\
                   'cl_pr_4h': clpr_4h, 'BB_UB_4h': '', 'SMA20_4h': '', 'BB_LB_4h': '', 'cat_4h': '',\
                   'cl_pr_1d': clpr_1d, 'BB_UB_1d': '', 'SMA20_1d': '', 'BB_LB_1d': '', 'cat_1d': ''})

dfPred.set_index('dates_15m', inplace=True)

dfPred['BB_UB_15m'] = dfPred['cl_pr_15m'].rolling(20).mean() + 2*dfPred['cl_pr_15m'].rolling(20).std()
dfPred['BB_LB_15m'] = dfPred['cl_pr_15m'].rolling(20).mean() - 2*dfPred['cl_pr_15m'].rolling(20).std()
dfPred['SMA20_15m'] = dfPred['cl_pr_15m'].rolling(window=20).mean()

dfPred['BB_UB_1h'] = dfPred['cl_pr_1h'].rolling(20).mean() + 2*dfPred['cl_pr_1h'].rolling(20).std()
dfPred['BB_LB_1h'] = dfPred['cl_pr_1h'].rolling(20).mean() - 2*dfPred['cl_pr_1h'].rolling(20).std()
dfPred['SMA20_1h'] = dfPred['cl_pr_1h'].rolling(window=20).mean()

dfPred['BB_UB_4h'] = dfPred['cl_pr_4h'].rolling(20).mean() + 2*dfPred['cl_pr_4h'].rolling(20).std()
dfPred['BB_LB_4h'] = dfPred['cl_pr_4h'].rolling(20).mean() - 2*dfPred['cl_pr_4h'].rolling(20).std()
dfPred['SMA20_4h'] = dfPred['cl_pr_4h'].rolling(window=20).mean()

#
dfPred['BB_UB_1d'] = dfPred['cl_pr_1d'].rolling(20).mean() + 2*dfPred['cl_pr_1d'].rolling(20).std()
dfPred['BB_LB_1d'] = dfPred['cl_pr_1d'].rolling(20).mean() - 2*dfPred['cl_pr_1d'].rolling(20).std()
dfPred['SMA20_1d'] = dfPred['cl_pr_1d'].rolling(window=20).mean()

# 15 Minute categorical variable 
dfPred.loc[(dfPred.cl_pr_15m > dfPred.BB_UB_15m),'cat_15m']='1'
dfPred.loc[((dfPred.cl_pr_15m > dfPred.SMA20_15m) & (dfPred.cl_pr_15m < dfPred.BB_UB_15m)),'cat_15m']='2'
dfPred.loc[((dfPred.cl_pr_15m < dfPred.SMA20_15m) & (dfPred.cl_pr_15m > dfPred.BB_LB_15m)),'cat_15m']='3'
dfPred.loc[(dfPred.cl_pr_15m < dfPred.BB_LB_15m),'cat_15m']='4'

# 1 Hour categorical variable 
dfPred.loc[(dfPred.cl_pr_1h > dfPred.BB_UB_1h),'cat_1h']='1'
dfPred.loc[((dfPred.cl_pr_1h > dfPred.SMA20_1h) & (dfPred.cl_pr_1h < dfPred.BB_UB_1h)),'cat_1h']='2'
dfPred.loc[((dfPred.cl_pr_1h < dfPred.SMA20_1h) & (dfPred.cl_pr_1h > dfPred.BB_LB_1h)),'cat_1h']='3'
dfPred.loc[(dfPred.cl_pr_1h < dfPred.BB_LB_1h),'cat_1h']='4'

# 4 Hour categorical variable 
dfPred.loc[(dfPred.cl_pr_4h > dfPred.BB_UB_4h),'cat_4h']='1'
dfPred.loc[((dfPred.cl_pr_4h > dfPred.SMA20_4h) & (dfPred.cl_pr_4h < dfPred.BB_UB_4h)),'cat_4h']='2'
dfPred.loc[((dfPred.cl_pr_4h < dfPred.SMA20_4h) & (dfPred.cl_pr_4h > dfPred.BB_LB_4h)),'cat_4h']='3'
dfPred.loc[(dfPred.cl_pr_4h < dfPred.BB_LB_4h),'cat_4h']='4'

# 1 Day categorical variable 
dfPred.loc[(dfPred.cl_pr_1d > dfPred.BB_UB_1d),'cat_1d']='1'
dfPred.loc[((dfPred.cl_pr_1d > dfPred.SMA20_1d) & (dfPred.cl_pr_1d < dfPred.BB_UB_1d)),'cat_1d']='2'
dfPred.loc[((dfPred.cl_pr_1d < dfPred.SMA20_1d) & (dfPred.cl_pr_1d > dfPred.BB_LB_1d)),'cat_1d']='3'
dfPred.loc[(dfPred.cl_pr_1d < dfPred.BB_LB_1d),'cat_1d']='4'

dfPred = dfPred.dropna()


In [ ]:
timeLastUpdate = 1639134000000 / 1000
print(datetime.fromtimestamp(timeLastUpdate))

2021-12-10 11:00:00


In [ ]:
def output(pred):
  if pred[-1] == '1':
    print("Our models are predicting now is a very opertune time to sell")
  elif pred[-1] == '2':
    print("Our models are predicting now is a time to weak sell or hold signal")
  elif pred[-1] == '3':
    print("Our models are predicting now is a time to weak buy or hold signal")
  elif pred[-1] == '4':
    print("Our models are predicting now is a very opertune time to buy")

In [ ]:
#Prediction:


dfPred = dfPred.dropna()
X =dfPred.drop(['cat_4h'],axis=1)

yhatDCT = dct.predict(X)
print('Decision Tree Prediction:')
output(yhatDCT)
yhatSVM = svmL.predict(X)
print('SVM Linear Kernal Prediction:')
output(yhatSVM)
yhatADB = adb.predict(X)
print('AdaBoost Prediction:')
output(yhatADB)



['3']
Decision Tree Prediction:
Our models are predicting now is a time to weak buy or hold signal
SVM Linear Kernal Prediction:
Our models are predicting now is a very opertune time to buy
AdaBoost Prediction:
Our models are predicting now is a time to weak sell or hold signal


*MISC Offcuts and Alternative validation Sets:* 

1.   List item

1.   List item
2.   List item


2.   List item



In [ ]:
#Feature Selection:
from sklearn.ensemble import ExtraTreesClassifier

X=df.drop(['cat_4h'],axis=1)
y=df['cat_4h']
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers

[0.02905245 0.0450334  0.04660579 0.05109374 0.06358488 0.04600088
 0.04231183 0.04576533 0.05076234 0.10647343 0.11701295 0.05465774
 0.04717836 0.0567179  0.0426704  0.03863664 0.03908598 0.03910159
 0.03825438]


In [ ]:
#Cross Validation tests 
valid = []
for i in range(10):
  crossVAL = cross_val_score(dct, X, y, cv=10)
  valid.append(crossVAL)
print('Decision tree classification Cross Validation Score:', (np.mean(valid)))
valid = []



Decision tree classification Cross Validation Score: 0.5289535524920467


In [ ]:
for i in range(5):
  crossVAL = cross_val_score(adb, X_under,y_under, cv=10)
  valid.append(crossVAL)
print('ADABoost Cross Validation Score:', (np.mean(valid)))
valid = []

ADABoost Cross Validation Score: 0.7398741067195754


In [ ]:
for i in range(5):
  crossVAL = cross_val_score(svmL, X, y, cv=10)
  valid.append(crossVAL)
print('Linear SVM Cross Validation Score:', (np.mean(valid))) 

NameError: ignored

In [ ]:
#Early Naieve Bayes Model
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB, BernoulliNB
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
gnb = GaussianNB()
cnb = CategoricalNB()
mnb = MultinomialNB()
bnb = BernoulliNB()
#X=df.drop(['cat_15m', 'cat_1h', 'cat_4h', 'cat_1d', 'date', 'cl_pr_15m', 'cl_pr_1h', 'cl_pr_4h', 'cl_pr_1d', 'cl_pr_1d'  ],axis=1) # input variables 
X=df.drop(['cat_4h' ],axis=1)
y=df['cat_4h']
accS=[]
recallS=[]
percS=[]

for x in range(100):
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=1)
  mnb.fit(X_train, y_train)
  y_pred = mnb.predict(X_test)
  acc=accuracy_score(y_test, y_pred)
  rec=recall_score(y_test, y_pred, average='weighted')
  perc=precision_score(y_test, y_pred, average='weighted')
  accS.append(acc)
  recallS.append(rec)
  percS.append(perc)
print("Accuracy", np.mean(accS))
print("Recall", np.mean(recallS))
print("Percision", np.mean(percS))

Accuracy 0.4706906558328496
Recall 0.4706906558328496
Percision 0.622299127230588


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(criterion = 'entropy')
rfc.fit(X_train, y_train)
rfcTrainPred = rfc.predict(X_test)
print("Accuracy:",accuracy_score(y_test, rfcTrainPred))
print('Precision:', precision_score(y_test, rfcTrainPred, average='weighted'))
print("Recall", recall_score(y_test, rfcTrainPred, average='weighted'))

Accuracy: 0.9199071387115496
Precision: 0.9151148418662295
Recall 0.9199071387115496


In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
tree.plot_tree(dct)